In [41]:
import numpy as np
import tensorflow as tf

x = np.array([[1, 1]])
c = 0.1 * np.asarray([[0, 1, 2, 3]])
h = 0.1 * np.asarray([[0, 1]])

In [42]:
x.shape

(1, 2)

In [43]:
c.shape

(1, 4)

In [44]:
h.shape

(1, 2)

In [45]:
c.size # num units

4

In [46]:
def sigmoid_array(x):
     return 1 / (1 + np.exp(-x))
    
class lstmcell:
    def __init__(self, num, x_, c_, h_):
        self.x = x_ #input
        self.c = c_# 초기 state
        self.h = h_# 초기 output
        self.num = num

    
    def build(self):
        num_units = self.num #hidden layer 갯수

        args = np.concatenate((self.x, self.h), axis=1)

        out_size = 4 * num_units
        proj_size = args.shape[-1]

        weights = np.ones([proj_size, out_size]) * 0.5

        out = np.matmul(args, weights)

        bias = np.ones([out_size]) * 0.5

        concat = out + bias

        i, j, f, o = np.split(concat, 4, 1)

        g = np.tanh(j)#Cell 현재값
        
        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)#forget gate

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g

        new_h = np.tanh(new_c) * sigmoid_array(o)

        new_c = np.split(new_c, 1, 1)
        new_h = np.split(new_h, 1, 1)

        return new_h, new_c,

In [47]:
num_units = c.size

In [115]:
def propagation(num_units, data, new_state, new_output):
    cell = lstmcell(num_units, x, new_state, new_output)
    output, state = cell.build()
    new_output = np.asarray(output)
    new_state = np.asarray(state)
    new_output = new_output.reshape(1, num_units)
    new_state = new_state.reshape(1, num_units)
    return new_state, new_output

In [116]:
new_state, new_output = c, h
error = np.array([0, 0])
state_predict = np.array([0.88490995, 1.84762319])

for i in range(1, 101, 1):
    new_state, new_output = propagation(num_units, x, new_state, new_output)
    global states
    error = (state_predict.mean()**2 - new_state.mean()**2)
    cost = error/i
    print(i)
    print("cost :", cost.mean())
    print("current output",new_output)
    print("current state", new_state, "\n")

1
cost : 1.0693599518908377
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

2
cost : 0.3581421049841107
current output [[0.68164891 0.72144474 0.75568855 0.78494467]]
current state [[0.92629413 1.02380943 1.12132472 1.21884002]] 

3
cost : 0.2236202062273386
current output [[0.7021526  0.74166484 0.77557645 0.80447775]]
current state [[0.94631075 1.04446175 1.14261275 1.24076376]] 

4
cost : 0.16645393522811636
current output [[0.70445045 0.74393313 0.77780969 0.80667331]]
current state [[0.94850799 1.04673033 1.14495267 1.24317501]] 

5
cost : 0.13305245732628493
current output [[0.70470282 0.74418228 0.77805502 0.80691453]]
current state [[0.94874873 1.04697891 1.14520909 1.24343926]] 

6
cost : 0.11086694110646785
current output [[0.70473047 0.74420958 0.7780819  0.80694096]]
current state [[0.94877511 1.04700614 1.14523718 1.24346821]] 

7
cost : 0.09502785763644675
current output [[0.7047335  0.74421257

########################################

In [50]:
tf_x = tf.convert_to_tensor(x, np.float32)
tf_c = tf.convert_to_tensor(c, np.float32)
tf_h = tf.convert_to_tensor(h, np.float32)
tf_new_c = tf.convert_to_tensor(new_state, np.float32)
tf_new_h= tf.convert_to_tensor(new_output, np.float32)

In [51]:
sess = tf.Session()

In [52]:
ta_x = tf.TensorArray(size=c.size, dtype=tf.float32)
ta_c = tf.TensorArray(size=c.size, dtype=tf.float32)
ta_h = tf.TensorArray(size=c.size, dtype=tf.float32)
ta_new_c = tf.TensorArray(size=c.size, dtype=tf.float32)
ta_new_h = tf.TensorArray(size=c.size, dtype=tf.float32)

In [53]:
ta_x = ta_x.write(0, tf_x)
ta_c = ta_c.write(0, tf_c)
ta_h = ta_h.write(0, tf_h)
ta_new_c = ta_new_c.write(0, tf_new_c)
ta_new_h = ta_new_h.write(0, tf_new_h)

In [54]:
ta_c.read(0)

<tf.Tensor 'TensorArrayReadV3_4:0' shape=(1, 4) dtype=float32>

In [55]:
sess.run(ta_c.read(0))

array([[0. , 0.1, 0.2, 0.3]], dtype=float32)

In [56]:
sess.run(tf.global_variables_initializer())

In [66]:
lstmcell(4, sess.run(ta_x.read(0)), sess.run(ta_c.read(0)), sess.run(ta_h.read(0))).build()

([array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])],
 [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])

In [65]:
t = 0
s = 0
def cond(t, s):
    return t<10
def body(t, s):
    return t+1, s+t

sess.run(tf.while_loop(cond=cond, body=body, loop_vars=(t, s)))

(10, 45)

In [ ]:
t = 0
def cond(t, ta_x, ta_c, ta_h):
    return t<10
def body(t, ta_x, ta_c, ta_h):
    new_state, new_output = lstmcell(4, sess.run(ta_x.read(0)), sess.run(ta_c.read(0)), sess.run(ta_h.read(0))).build()
    return t+1, new_state, new_output

In [97]:
body(t, ta_x, ta_c, ta_h)

(1,
 [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])],
 [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])

In [ ]:
sess.run(tf.while_loop(cond=cond, body=body, loop_vars=(t, tf_x, tf_c, tf_h)))

In [101]:
for i in range(10):
    print(body(t, ta_x, ta_c, ta_h))

(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]])])
(1, [array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]])], [array([[0.7537942 , 0.84655155, 0.9393089 , 1.032066

#########################################

In [102]:
def propagation(num_units, data, new_state, new_output):
    cell = lstmcell(num_units, x, new_state, new_output)
    output, state = cell.build()
    new_output = np.asarray(output)
    new_state = np.asarray(state)
    new_output = new_output.reshape(1, num_units)
    new_state = new_state.reshape(1, num_units)
    return new_state, new_output

In [106]:
propagation(4, sess.run(ta_x.read(0)), sess.run(ta_c.read(0)), sess.run(ta_h.read(0)))

(array([[0.7537942 , 0.84655155, 0.9393089 , 1.03206625]]),
 array([[0.52580584, 0.56858303, 0.60623287, 0.63909013]]))

In [114]:
new_state, new_output = sess.run(ta_c.read(0)), sess.run(ta_h.read(0))
error = np.array([0, 0])
state_predict = np.array([0.88490995, 1.84762319])

for i in range(1, 101, 1):
    new_state, new_output = propagation(num_units, sess.run(ta_x.read(0)), sess.run(ta_c.read(0)), sess.run(ta_h.read(0)))
    global states
    error = (state_predict.mean()**2 - new_state.mean()**2)
    cost = error/i
    print(i)
    print("cost :", cost.mean())
    print("current output",new_output)
    print("current state", new_state, "\n")

1
cost : 1.0693599515207215
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

2
cost : 0.5346799757603607
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

3
cost : 0.3564533171735738
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

4
cost : 0.2673399878801804
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

5
cost : 0.2138719903041443
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

6
cost : 0.1782266585867869
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

7
cost : 0.15276570736010306
current output [[0.52580584 0.56858303 0.

54
cost : 0.019802962065198545
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

55
cost : 0.019442908209467664
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

56
cost : 0.019095713420012882
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

57
cost : 0.01876070090387231
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

58
cost : 0.018437240543460715
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

59
cost : 0.018124744941029176
current output [[0.52580584 0.56858303 0.60623287 0.63909013]]
current state [[0.7537942  0.84655155 0.9393089  1.03206625]] 

60
cost : 0.017822665858678692
current output [[0.525